In [1]:
import datetime
import pandas as pd
import glob
import time
import re
import os
import requests
import json
from bs4 import BeautifulSoup
from collections import Counter

In [2]:
class EdgarMasterDownload():
    
    """
    
    This class downloads the master files of Edgar filing for all companies to a specified folder
    
    The different between EdgarMasterDownload and EdgarMasterDownloadSelenium is that this class requires you
    to provide your information in User-Agent. Otherwise SEC will block you.
    
    """
    def __init__(self,user_agent,start_year = 1993, end_year = 'now',output_folder = 'edgar_master'):
        
        """
        
        Args:
            start_year: must be an integer
            end_year: must be an integer or the string 'now'
            output_folder: must be provided. Make sure it is a new folder without any file
            user_agent: this is a string that contains both your name and email address required by SEC.
                        For example, Your Name Your Email Address
        
        """
        
        
        self.start_year = start_year
        self.end_year = end_year
        self.output_folder = output_folder
        self.user_agent = user_agent
        self.headers = {'User-Agent':user_agent,
           'Accept-Encoding': 'gzip, deflate',
           'Host':'www.sec.gov'}
        
        if not os.path.exists(self.output_folder):
            os.makedirs(self.output_folder)
        
        if '/' not in self.output_folder:
            self.output_folder = os.getcwd()+'/'+self.output_folder
        
        if re.findall('^./',self.output_folder)!=[]:
            self.output_folder = re.sub('^./',os.getcwd()+'/',self.output_folder)
        
    def get_master_file_url(self):

        """
        Find all urls of master files
        """

        if self.end_year == 'now':
            end_year_int = datetime.date.today().year
            end_quarter_int = (datetime.date.today().month - 1) // 3 + 1
        else:
            end_year_int = int(self.end_year)
            end_quarter_int = 4
        years = list(range(self.start_year, end_year_int+(self.end_year != 'now')))
        quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']
        history = [(y, q) for y in years for q in quarters]

        if self.end_year == 'now':
            for i in range(1, end_quarter_int + 1):
                history.append((end_year_int, 'QTR%s' % i))
        urls = ['https://www.sec.gov/Archives/edgar/full-index/%s/%s/master.idx' % (x[0], x[1]) for x in history]
        urls.sort()
        print('There are {} urls'.format(len(urls)))
        return urls  
    
    def get_single_master_file(self,master_file_url):
        lines= requests.get(master_file_url,headers = self.headers).text.splitlines()
        col_name = re.sub('\s+','',lines[9]).split('|')
        master_file_df_temp = pd.DataFrame()
        master_file_df_temp[col_name[0]] = [re.sub('\s+','',x).split('|')[0] for x in lines[11:]]
        master_file_df_temp[col_name[1]] = [re.sub('\s+','',x).split('|')[1] for x in lines[11:]]
        master_file_df_temp[col_name[2]] = [re.sub('\s+','',x).split('|')[2] for x in lines[11:]]
        master_file_df_temp[col_name[3]] = [re.sub('\s+','',x).split('|')[3] for x in lines[11:]]
        master_file_df_temp[col_name[4]] = [re.sub('\s+','',x).split('|')[4] for x in lines[11:]]
        return master_file_df_temp
     
    def get_master_file_combined(self,return_df = False):
        urls = self.get_master_file_url()
        master_file_df_list = []
        for url in urls:
            master_file_df_list.append(self.get_single_master_file(url))
        master_file_df = pd.concat(master_file_df_list)
        
        master_file_df.to_csv(self.output_folder+'/master_file_df.csv', index= False)
        
        if return_df == True:
            return master_file_df
        else:
            return None

In [ ]:
edgar_master_download = EdgarMasterDownload('James Jiang qiaojiang0114@gmail.com',start_year = 1993, end_year = 'now',output_folder = 'edgar_master')

In [ ]:
master_file_df = edgar_master_download.get_master_file_combined(return_df = True)

In [3]:
master_file_df = pd.read_csv('edgar_master/master_file_df.csv')

In [4]:
master_file_df.shape

(22959799, 5)

In [6]:
master_file_df[master_file_df['CIK']==320193].shape

(1965, 5)

In [8]:
master_file_df[master_file_df['CIK']==320193].tail()

,CIK,CompanyName,FormType,DateFiled,Filename
22888246,320193,AppleInc.,PX14A6G,2023-02-22,edgar/data/320193/0001214659-23-002862.txt
22888247,320193,AppleInc.,SC13G/A,2023-02-07,edgar/data/320193/0001306550-23-008266.txt
22888248,320193,AppleInc.,SC13G/A,2023-02-09,edgar/data/320193/0001104659-23-015055.txt
22888249,320193,AppleInc.,SC13G/A,2023-02-14,edgar/data/320193/0001193125-23-038262.txt
22888250,320193,AppleInc.,SD,2023-03-16,edgar/data/320193/0001140361-23-012065.txt


In [10]:
x = 'edgar/data/320193/0001214659-23-002862.txt'

In [13]:
def convert_to_url(x):
    x = 'https://www.sec.gov/Archives/'+x.replace('-','').replace('.txt','')
    return x

In [16]:
convert_to_url('edgar/data/320193/0001193125-23-038262.txt')

'https://www.sec.gov/Archives/edgar/data/320193/000119312523038262'

In [5]:
master_file_df.head()

,CIK,CompanyName,FormType,DateFiled,Filename
0,860585,RBSPARTNERSLP/CT,13FCONP,1993-02-11,edgar/data/860585/9999999997-04-035713.txt
1,880794,MERRILLLYNCHLIFEVARIABLEANNUITYSEPARATEACCOUNTB,NSAR-B,1993-02-26,edgar/data/880794/9999999997-05-050433.txt
2,926688,SMITHTHOMASW,13F-HR,1993-02-12,edgar/data/926688/9999999997-05-015654.txt
3,94673,STORAGETECHNOLOGYCORP,CERTNYS,1993-02-24,edgar/data/94673/9999999997-05-037760.txt
4,860585,RBSPARTNERSLP/CT,13FCONP,1993-05-12,edgar/data/860585/9999999997-04-042068.txt


In [21]:
company_wanted_df = pd.read_stata('Deals_with_Public_Target_Ticker&CIK.dta')
cik_cusip_linking_table = pd.read_csv('edgar_master/cik-cusip-maps.csv')
cik_ticker_linking_table = pd.read_csv('cik_ticker_linking_table.csv')

cik_cusip_linking_table['cik'] = [str(int(float(x))) for x in cik_cusip_linking_table['cik']]
company_wanted_df['acquirer_cik'] = [str(int(x)) for x in company_wanted_df['acquirer_cik']]
cik_ticker_linking_table['cik'] = [str(int(x)) for x in cik_ticker_linking_table['cik']]

In [31]:
company_wanted_df[company_wanted_df['acquirer_cik']=='']

,deal_number,date_announced,acquirer_cik,date_effective,acquirer_cusip,acquirer_ticker_symbol,target_cusip,target_ticker_symbol,target_public_status,acquirer_name,target_name,target_cik,target_ticker_crsp


In [7]:
master_file_cik = list(master_file_df['CIK'].unique())

In [23]:
master_file_cik = [str(int(x)) for x in master_file_cik]

In [24]:
len(master_file_cik)

835225

In [25]:
company_wanted_df['acquirer_cik'].nunique()

440

In [26]:
master_file_df.head(2)

,CIK,CompanyName,FormType,DateFiled,Filename
0,860585,RBSPARTNERSLP/CT,13FCONP,1993-02-11,edgar/data/860585/9999999997-04-035713.txt
1,880794,MERRILLLYNCHLIFEVARIABLEANNUITYSEPARATEACCOUNTB,NSAR-B,1993-02-26,edgar/data/880794/9999999997-05-050433.txt


In [39]:
def find_cik(cik,cusip,ticker,master_file_cik):
    cik_new = 'unknown'
    source = 'unknown'
    if (cik!='') and (cik in master_file_cik):
        cik_new = cik
        source = 'cik'
    else:
        cik_matched = cik_cusip_linking_table[cik_cusip_linking_table['cusip6']==cusip]['cik'].tolist()
        if (len(cik_matched)!=0) and (cik_matched[0] in master_file_cik):
            cik_new = cik_matched[0]
            source = 'cusip'
        else:
            cik_matched = cik_ticker_linking_table[cik_ticker_linking_table['ticker']==ticker]['cik'].tolist()
            if (len(cik_matched)!=0) and (cik_matched[0] in master_file_cik):
                cik_new = cik_matched[0]
                source = 'ticker'
    return cik_new, source

In [40]:
master_file_df['CIK'] = [str(int(x)) for x in master_file_df['CIK']]

In [41]:
def download_use_cik_acquirer(cik,cusip,ticker,master_file_cik):
    time_break = 0.5
    headers = {'User-Agent':'Qiao Jiang qiaojiang0114@gmail.com',
           'Accept-Encoding': 'gzip, deflate',
           'Host':'www.sec.gov'}
    acquirer_cik = cik
    cik, source = find_cik(cik,cusip,ticker,master_file_cik)
    if cik == 'unknown':
        return None
    temp = master_file_df[master_file_df['CIK']==cik]
    temp.reset_index(inplace=True, drop = True)
    print(f'Use CIK information. Download starts! There are {temp.shape[0]} files to download.')
    for i in range(temp.shape[0]):
        cik_temp = temp.loc[i,'CIK']
        FormType_temp = temp.loc[i,'FormType']
        FormType_temp = FormType_temp.replace('/','#')
        DateFiled = temp.loc[i,'DateFiled']
        Filename = temp.loc[i,'Filename']
        
        if not os.path.exists('edgar_master/acquirer_'+acquirer_cik):
            os.makedirs('edgar_master/acquirer_'+acquirer_cik)
        file_name = 'edgar_master/acquirer_'+acquirer_cik+'/'+'_'.join([cik_temp,FormType_temp,DateFiled,cik,cusip,ticker])+'.txt'
        r = requests.get('https://www.sec.gov/Archives/'+Filename, allow_redirects=True, headers = headers)
        if r.status_code == 200:
            with open(file_name, "wb") as f:
                f.write(r.content)
                #print("File saved successfully!")
        else:
            print("Failed to download file")
        time.sleep(time_break)
    print('All files saved successfully!')
    result = pd.DataFrame({'cik':[acquirer_cik],
                           'cusip':[cusip],
                           'status':['success'],
                           'identifier':[source]})
    result.to_csv('edgar_master/acquirer_'+acquirer_cik+'/'+'download_status.csv', index = False)

In [ ]:
for i, acquirer_cik_cusip_ticker in enumerate(company_wanted_df[['acquirer_cik','acquirer_cusip','acquirer_ticker_symbol']].to_numpy()):
    cik_result_files = glob.glob('edgar_master/acquirer_'+acquirer_cik_cusip_ticker[0]+'/download_status.csv') 
    if len(cik_result_files)==0:
        print(i)
        download_use_cik_acquirer(*acquirer_cik_cusip_ticker,master_file_cik)